In [2]:
!pip install -q lxml

In [32]:
import bs4
import lxml
import pandas as pd
import urllib
import numpy

In [7]:
from urllib import request

In [119]:
def firefoxPage(link):
    req = request.Request(
        url=link, 
        headers={'User-Agent': 'Mozilla/5.0'} # IMPORTANT (sinon connection refusee)
    )
    html_page = request.urlopen(req)
    return html_page

### Obtenir les liens de tous les joueurs en LCK summer 2023

In [180]:
from bs4 import BeautifulSoup

html_page = firefoxPage('https://gol.gg/players/list/season-ALL/split-ALL/tournament-LCK%20Summer%202023/')
soup = BeautifulSoup(html_page, features="lxml")
player_link_list = []
for link in soup.find_all('a'):
    if str(link.get('href')).startswith("./player-stats/"):
        player_link_list.append(str(link.get('href')).strip('.'))

### Obtenir les stats des joueurs

In [181]:
def checkCols(cols):
    if cols == None:
        return False
    for k in cols:
        if k != '':
            return True
    return False

data = []
columnNames = ['Player']
generateCol = True
for link in player_link_list:
    link = link.strip()
    html = firefoxPage("https://gol.gg/players" + link)
    soup = BeautifulSoup(html, features="lxml")

    table = soup.find("table", attrs={"class": "table_list"})
    name = soup.find("h1").text.strip()
    playerStat = [name]

    rows = table.find_all('tr')
    if columnNames != ['Player']:
        generateCol = False
    for row in rows:

        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]

        if checkCols(cols):
            if generateCol:
                columnNames.append(cols[0])
            playerStat = playerStat + [elem for elem in cols[1:] if elem != '']
    data.append(playerStat)

df = pd.DataFrame(data, columns=columnNames)

In [182]:
print(df)

      Player    Record: Win Rate: KDA: CS per Minute: Gold Per Minute: Gold%:  \
0     Aiming   34W - 6L       85%  9.1           10.5              508    26%   
1      Andil   2W - 18L       10%  1.3            1.1              215  12.7%   
2        Bdd   34W - 6L       85%  7.2            8.8              417  21.3%   
3      BeryL  16W - 27L     37.2%  2.1            1.3              232  13.5%   
4      Bible    8W - 5L     61.5%  2.8            1.1              238  12.9%   
..       ...        ...       ...  ...            ...              ...    ...   
61    Willer  13W - 27L     32.5%  3.0            5.6              330  18.6%   
62  YoungJae  10W - 28L     26.3%  2.6            5.7              321  18.4%   
63      Zeka  27W - 14L     65.9%  5.2            9.4              436    23%   
64      Zeus  21W - 21L       50%  3.0            8.4              388  20.9%   
65    kingen  27W - 14L     65.9%  3.7            8.4              391  20.7%   

   Kill Participation:  
0 

### Obtenir les liens de tous les equipes

In [135]:
from bs4 import BeautifulSoup

html_page = firefoxPage('https://gol.gg/teams/list/season-S13/split-Summer/tournament-LCK%20Summer%202023/')
soup = BeautifulSoup(html_page, features="lxml")
team_link_list = []
for link in soup.find_all('a'):
    if str(link.get('href')).startswith("./team-stats/"):
        team_link_list.append(str(link.get('href')).strip('.').replace('LCK Summer 2023/', 'LCK%20Summer%202023/'))

### Obtenir les roles des joueurs

In [ ]:
BRION player's stats

In [201]:
labels = ['Team','Role', 'Player']
data = []

def get_player_table(soup, team_name):
    tables = soup.find_all('table')
    for table in tables:
        caption = table.find('caption')
        if caption and caption.text.strip() == team_name + ' player\'s stats':
            return table
        else:
            continue
    return None

for link in team_link_list:
    link = link.strip()
    html = firefoxPage("https://gol.gg/teams" + link)
    soup = BeautifulSoup(html, features="lxml")

    table = soup.find("table", attrs={"class": "table_list"})
    name = soup.find("h1").text.strip()
    rows = get_player_table(soup, name).find_all('tr')
    
    for row in rows:
        tds = row.find_all('td')
        if len(tds) >= 2:
            stat = [name]
            stat.append(tds[0].text.strip())
            stat.append(tds[1].text.strip())
            data.append(stat)
dfrole = pd.DataFrame(data, columns=labels)

[['BRION', 'TOP', 'Morgan'], ['BRION', 'JUNGLE', 'UmTi'], ['BRION', 'MID', 'Karis'], ['BRION', 'BOT', 'Hena'], ['BRION', 'SUPPORT', 'Effort'], ['BRION', 'MID', 'Ivory'], ['Dplus KIA', 'TOP', 'Canna'], ['Dplus KIA', 'JUNGLE', 'Canyon'], ['Dplus KIA', 'MID', 'ShowMaker'], ['Dplus KIA', 'BOT', 'Deft'], ['Dplus KIA', 'SUPPORT', 'Bible'], ['Dplus KIA', 'BOT', 'Rahel'], ['Dplus KIA', 'SUPPORT', 'Kellin'], ['DRX', 'TOP', 'Rascal'], ['DRX', 'JUNGLE', 'Croco'], ['DRX', 'MID', 'FATE'], ['DRX', 'BOT', 'Paduck'], ['DRX', 'SUPPORT', 'BeryL'], ['DRX', 'JUNGLE', 'Juhan'], ['DRX', 'MID', 'Kyeahoo'], ['Gen.G eSports', 'TOP', 'Doran'], ['Gen.G eSports', 'JUNGLE', 'Peanut'], ['Gen.G eSports', 'MID', 'Chovy'], ['Gen.G eSports', 'BOT', 'Peyz'], ['Gen.G eSports', 'SUPPORT', 'Delight'], ['Hanwha Life eSports', 'TOP', 'kingen'], ['Hanwha Life eSports', 'JUNGLE', 'Grizzly'], ['Hanwha Life eSports', 'MID', 'Zeka'], ['Hanwha Life eSports', 'BOT', 'Viper'], ['Hanwha Life eSports', 'SUPPORT', 'Life'], ['Hanwha Lif

In [176]:
print(dfrole)

     Team     Role    Player
0   BRION      TOP    Morgan
1   BRION   JUNGLE      UmTi
2   BRION      MID     Karis
3   BRION      BOT      Hena
4   BRION  SUPPORT    Effort
..    ...      ...       ...
61     T1   JUNGLE      Oner
62     T1      MID     Faker
63     T1      BOT  Gumayusi
64     T1  SUPPORT     Keria
65     T1      MID      Poby

[66 rows x 3 columns]


### Stats finaux joueurs

In [183]:
players = pd.merge(dfrole, df)

In [184]:
print(players)

     Team     Role    Player    Record: Win Rate: KDA: CS per Minute:  \
0   BRION      TOP    Morgan  15W - 30L     33.3%  2.1            8.9   
1   BRION   JUNGLE      UmTi  15W - 30L     33.3%  2.9            5.4   
2   BRION      MID     Karis  12W - 23L     34.3%  3.5            9.2   
3   BRION      BOT      Hena  15W - 30L     33.3%  3.3             10   
4   BRION  SUPPORT    Effort  15W - 30L     33.3%  2.6              1   
..    ...      ...       ...        ...       ...  ...            ...   
61     T1   JUNGLE      Oner  21W - 21L       50%  3.5            5.6   
62     T1      MID     Faker   17W - 7L     70.8%  3.4            8.8   
63     T1      BOT  Gumayusi  21W - 21L       50%  4.6           10.7   
64     T1  SUPPORT     Keria  21W - 21L       50%  3.6            1.1   
65     T1      MID      Poby   4W - 14L     22.2%  1.6              9   

   Gold Per Minute: Gold%: Kill Participation:  
0               390  22.3%                 52%  
1               317  18.1

### Obtenir stats equipes

In [203]:
from bs4 import BeautifulSoup

html_page = firefoxPage('https://gol.gg/teams/list/season-S13/split-Summer/tournament-LCK%20Summer%202023/')
soup = BeautifulSoup(html_page, features="lxml")

table = soup.find_all('table')[1]
rows = table.find_all('tr')

labels = [elem.text.strip() for elem in table.find('thead').find_all('th')]
data = []
for row in rows:
    stat = []
    tds = row.find_all('td')
    for td in tds:
        stat.append(td.text.strip())
    if (stat != []):
        data.append(stat)

dfteam  = pd.DataFrame(data, columns=labels)

In [204]:
print(dfteam)

                  Name Season Region Games Win rate   K:D   GPM   GDM  \
0                BRION    S13     KR    45    33.3%  0.79  1749  -112   
1            Dplus KIA    S13     KR    40    60.0%  1.23  1859    78   
2                  DRX    S13     KR    43    37.2%  0.72  1733  -159   
3        Gen.G eSports    S13     KR    39    82.1%  1.77  1969   275   
4  Hanwha Life eSports    S13     KR    41    65.9%  1.27  1887    98   
5           KT Rolster    S13     KR    41    85.4%  1.94  1957   248   
6     Kwangdong Freecs    S13     KR    38    26.3%  0.61  1746  -164   
7         Liiv SANDBOX    S13     KR    40    32.5%  0.82  1784  -106   
8    Nongshim RedForce    S13     KR    43    30.2%  0.70  1739  -190   
9                   T1    S13     KR    42    50.0%  0.97  1860    60   

  Game duration Kills / game  ... TD@15  GD@15   PPG NASHPG NASH%   CSM   DPM  \
0         32:17          8.4  ...   0.0   -277  3.47   0.60  43.8  34.5  1848   
1         31:38         13.1  ... 